In [1]:
from models.generator import SEMamba
import torch
from utils.util import load_config

In [2]:
print('Initializing Inference Process..')
# 修改 config 為有效的 YAML 配置文件路徑
config = '/disk4/chocho/SEMamba/exp/20250403-SEMamba_v1_PCS-h32-tf2/config.yaml'  # 替換為實際的 YAML 檔案路徑
checkpoint_file = '/disk4/chocho/SEMamba/exp/20250403-SEMamba_v1_PCS-h32-tf2/g_00078000.pth'

global device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    #device = torch.device('cpu')
    raise RuntimeError("Currently, CPU mode is not supported.")

cfg = load_config(config)
model = SEMamba(cfg).to(device)
state_dict = torch.load(checkpoint_file, map_location=device)
model.load_state_dict(state_dict['generator'])

Initializing Inference Process..


<All keys matched successfully>

In [3]:
import torch
import torch.onnx

# 假設你的模型是 `model`
model.eval()

SEMamba(
  (dense_encoder): DenseEncoder(
    (dense_conv_1): Sequential(
      (0): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): PReLU(num_parameters=32)
    )
    (dense_block): DenseBlock(
      (dense_block): ModuleList(
        (0): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (2): PReLU(num_parameters=32)
        )
        (1): Sequential(
          (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 1), dilation=(2, 1))
          (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (2): PReLU(num_parameters=32)
        )
        (2): Sequential(
          (0): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(4, 1), dilation=(4, 1))
          (1): 

In [4]:
cd _202505

/disk4/chocho/SEMamba/_202505


In [6]:
model.dense_encoder.dense_conv_1[0]

Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1))

In [5]:
model.TSMamba[0].time_mamba.forward_blocks[0]

Block(
  (mixer): Mamba(
    (in_proj): Linear(in_features=32, out_features=256, bias=False)
    (conv1d): Conv1d(128, 128, kernel_size=(4,), stride=(1,), padding=(3,), groups=128)
    (act): SiLU()
    (x_proj): Linear(in_features=128, out_features=34, bias=False)
    (dt_proj): Linear(in_features=2, out_features=128, bias=True)
    (out_proj): Linear(in_features=128, out_features=32, bias=False)
  )
  (norm): RMSNorm()
)

In [7]:
# 定義輸入張量的形狀 (dummy input)
shape = 1,2,286,201
print("B,L,D:", shape)
dummy_input = torch.randn(shape).to("cuda")
# dummy_input = (torch.randn(batch, length, dim).to("cuda"),
            #    torch.randn(batch, length, dim).to("cuda"))

# 導出 ONNX 模型
torch.onnx.export(
    model.dense_encoder.dense_conv_1[0],
    dummy_input,               # 輸入範例
    "onnx_files/dense_encoder-dense_conv_1.onnx",              # 輸出檔案名稱
    input_names=["input"],     # 輸入層名稱
    output_names=["output"],   # 輸出層名稱
    dynamic_axes={             # 支援動態形狀（可選）
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=11           # ONNX 操作集版本，常用 11 或更高
)

B,L,D: (1, 2, 286, 201)


In [23]:
# 定義輸入張量的形狀 (dummy input)
shape = 100,286,32
print("B,L,D:", shape)
dummy_input = torch.randn(shape).to("cuda")
# dummy_input = (torch.randn(batch, length, dim).to("cuda"),
            #    torch.randn(batch, length, dim).to("cuda"))

# 導出 ONNX 模型
torch.onnx.export(
    model.TSMamba[0].time_mamba.forward_blocks[0].mixer,
    dummy_input,               # 輸入範例
    "onnx_files/time_mamba-Mamba.onnx",              # 輸出檔案名稱
    input_names=["input"],     # 輸入層名稱
    output_names=["output"],   # 輸出層名稱
    dynamic_axes={             # 支援動態形狀（可選）
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=11           # ONNX 操作集版本，常用 11 或更高
)

B,L,D: (100, 286, 32)


In [ ]:
# 定義輸入張量的形狀 (dummy input)
shape = 100,286,32
print("B,L,D:", shape)
dummy_input = torch.randn(shape).to("cuda")

# 導出 ONNX 模型
torch.onnx.export(
    model.TSMamba[0].time_mamba.forward_blocks[0].norm,
    dummy_input,               # 輸入範例
    "onnx_files/time_mamba-RMSNorm.onnx",              # 輸出檔案名稱
    input_names=["input"],     # 輸入層名稱
    output_names=["output"],   # 輸出層名稱
    dynamic_axes={             # 支援動態形狀（可選）
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=11           # ONNX 操作集版本，常用 11 或更高
)

B,L,D: (100, 286, 32)


/disk4/chocho/speechbrain/.speechbrain/lib/python3.11/site-packages/mamba_ssm/ops/triton/layernorm.py:409: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  else (torch.float32 if residual_in_fp32 else None)
/disk4/chocho/speechbrain/.speechbrain/lib/python3.11/site-packages/mamba_ssm/ops/triton/layernorm.py:133: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert weight.shape == (N,)
/disk4/chocho/speechbrain/.speechbrain/lib/python3.11/site-packages/mamba_ssm/ops/triton/layernorm.py:150: TracerWarning: Converting a tensor to a Python boolean might cause

CompilationError: at 31:24:    HAS_BIAS: tl.constexpr,
):
    # Map the program id to the row of X and Y it should compute.
    row = tl.program_id(0)
    X += row * stride_x_row
    Y += row * stride_y_row
    if HAS_RESIDUAL:
        RESIDUAL += row * stride_res_row
    if STORE_RESIDUAL_OUT:
        RESIDUAL_OUT += row * stride_res_out_row
    # Compute mean and variance
    cols = tl.arange(0, BLOCK_N)
                        ^
ValueError("arange's arguments must be of type tl.constexpr")

In [ ]:
# 定義輸入張量的形狀 (dummy input)
shape = 100,286,32
print("B,L,D:", shape)
dummy_input = torch.randn(shape).to("cuda")
# dummy_input = (torch.randn(batch, length, dim).to("cuda"),
            #    torch.randn(batch, length, dim).to("cuda"))

# 導出 ONNX 模型
torch.onnx.export(
    model.TSMamba[0].time_mamba.forward_blocks[0].mixer,
    dummy_input,               # 輸入範例
    "onnx_files/time_mamba-Mamba.onnx",              # 輸出檔案名稱
    input_names=["input"],     # 輸入層名稱
    output_names=["output"],   # 輸出層名稱
    dynamic_axes={             # 支援動態形狀（可選）
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=11           # ONNX 操作集版本，常用 11 或更高
)